In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

In [2]:
def get_currencies(currencies, start, end, export_csv = False):
    frames = []
    for currency in currencies:
        while True:
            try:
                url = f'https://www.investing.com/currencies/krw-{currency.lower()}-historical-data'
                option = Options()
                option.headless = False
                driver = webdriver.Chrome(options=option)
                driver.get(url)
                driver.maximize_window()
    
                date_button = WebDriverWait(driver,20).until(EC.element_to_be_clickable(
                    (By.XPATH,'/html/body/div[5]/section/div[8]/div[3]/div/div[2]/span')))
                date_button.click()
    
                start_bar = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(
                    (By.XPATH,'/html/body/div[7]/div[1]/input[1]')))
                start_bar.clear()
                start_bar.send_keys(start)
    
                end_bar = WebDriverWait(driver,20).until(EC.element_to_be_clickable(
                    (By.XPATH,'/html/body/div[7]/div[1]/input[2]')))
                end_bar.clear()
                end_bar.send_keys(end)
    
                apply_button = WebDriverWait(driver,20).until(EC.element_to_be_clickable(
                    (By.XPATH,'/html/body/div[7]/div[5]/a')))
                apply_button.click()
                sleep(5)
    
                dataframes = pd.read_html(driver.page_source)
                driver.quit()
                print(f'{currency} scraped.')
                break
            
            except:
                driver.quit()
                print(f'Failed to scrape {currency}. Trying again in 10 seconds')
                sleep(10)
                continue
            
        for dataframe in dataframes:
            if dataframe.columns.tolist() == ['Date','Price','Open','High','Low','Change%']:
                df = dataframe
                frames.append(df)
            if export_csv:
                dataframe.to_csv(f'{currency}.csv', index=False)
                print(f'{currency}.csv exported.')
                break

In [3]:
currencies = ['eur','jpy','chf']
get_currencies(currencies,start='01/01/2020',end='01/14/2021',export_csv=True)

eur scraped.
eur.csv exported.
jpy scraped.
jpy.csv exported.
chf scraped.
chf.csv exported.
